In [96]:
# opt = TestOptions().parse()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = UNet(n_channels=1, n_classes=6).to(device)

In [97]:
import torch
from torch.utils.data import DataLoader
from unet import UNet
from options.test_options import TestOptions

In [98]:
os.listdir('/home/omen/Documents/Nikita/GenSeg/checkpoint_bhsd')

['fewshot_10',
 '20250721-133152-bhsd',
 '20250721-135513-bhsd',
 '20250721-151833-bhsd',
 '20250721-162837-bhsd',
 '20250721-134307-bhsd',
 '20250721-223914-bhsd',
 '20250721-132920-bhsd']

In [99]:

net.load_state_dict(torch.load("/home/omen/Documents/Nikita/GenSeg/checkpoint_bhsd/20250721-162837-bhsd/unet.pth",  map_location=device))

<All keys matched successfully>

In [11]:
import os
os.listdir("/home/omen/Documents/Nikita/GenSeg/checkpoint_bhsd/20250721-162837-bhsd/unet.pth")

['unet.pth']

In [9]:
checkpoint_bhsd/fewshot_10/20250721-232647-bhsd


'/home/omen/Documents/Nikita/GenSeg'

In [100]:
Dir = '/home/omen/Documents/Rishabh/TrainingDataApp/TrainingData'
MasksPath = os.path.join(Dir, 'Masks')
ImagesPath = os.path.join(Dir, 'Images')
os.listdir(Dir)

['Masks', 'Images']

In [101]:
datadict = {'BackGround' :0, 
            'Bleed-Subdural': 1,  
            'Scalp-Hematoma' :2, 
            'Bleed-Others':3, 
            'Bleed-Intraventricular':4,  
            'Bleed-Epidural': 5 , 
            'Bleed-Contusion': 6, 
            'Bleed-Hematoma' :7, 
            'Bleed-Subarachnoid': 8}

reversed_dict = {v: k for k, v in datadict.items()}

In [102]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, datadict=datadict):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        self.series = os.listdir(mask_dir)
        self.datadict = datadict
        reversed_dict = {v: k for k, v in datadict.items()}
        self.reversed_dict = reversed_dict
        self.clip_min = 0
        self.clip_max = 255
        self.scale = 1
        
    def __len__(self):
        return len(self.series)


    def _preprocess_img(self, slice2d: np.ndarray) -> np.ndarray:
        # Clip to soft-tissue window and normalize to [0, 1]
        slice2d = np.clip(slice2d, self.clip_min, self.clip_max)
        slice2d = (slice2d - self.clip_min) / (self.clip_max - self.clip_min)
        slice2d = slice2d.astype(np.float32)
        if scale != 1.0:
            h, w = slice2d.shape
            slice2d = cv2.resize(slice2d, (int(w * self.scale), int(h *self.scale)), interpolation=cv2.INTER_LINEAR)
        # Add channel dimension (C, H, W) with C=1
        return slice2d[np.newaxis, ...]

    def _preprocess_mask(self, mask2d: np.ndarray) -> np.ndarray:
        # print('mask before interpolation:-' , np.unique(mask2d))
        if self.scale != 1.0:
            h, w = mask2d.shape
            mask2d = cv2.resize(mask2d.astype(np.uint8), (int(w * self.scale), int(h * self.scale)), interpolation=cv2.INTER_NEAREST)
        mask2d = mask2d.astype(np.int64)
        # print('mask after interpolation:-' , np.unique(mask2d))
        return mask2d[np.newaxis, ...]  # keep channel dim for compatibility

    def __getitem__(self, index):
        Maskvolume = []
        ImageVolume = []
        print(self.series[index])
        flag = 0
        for key in range(len(self.reversed_dict.keys())):
            catag = self.reversed_dict[key]
            Maskcatgvolume = []
            Masks = os.path.join(self.mask_dir, os.listdir(self.mask_dir)[index], catag)
            MasksList = os.listdir(Masks)
            MasksList = sorted(MasksList)
            
            for msk in MasksList:
                pngMask = Image.open(os.path.join(Masks, msk))
                pngMask = np.array(pngMask)
                Maskcatgvolume.append(pngMask)
    
                if msk in self.images and flag == 0:
                    pngimage = Image.open(os.path.join(self.image_dir ,msk))
                    pngimage = np.array(pngimage)
                    ImageVolume.append(pngimage)
            flag = 1
                    
            Maskcatgvolume = np.stack(Maskcatgvolume, axis = 0)
            Maskvolume.append(Maskcatgvolume)
            
        Maskvolume = np.stack(Maskvolume, axis = 0)
        ImageVolume = np.stack(ImageVolume, axis = 0)
        NewMaskVolume = []
        NewMaskVolume.append(Maskvolume[0,:,:,:])
        NewMaskVolume.append(Maskvolume[5,:,:,:])
        Maskvolume[6,:,:,:] = Maskvolume[6,:,:,:] + Maskvolume[7,:,:,:]
        Maskvolume[6,:,:,:][Maskvolume[6,:,:,:] > 0] = 1
        NewMaskVolume.append(Maskvolume[6,:,:,:])
        NewMaskVolume.append(Maskvolume[4,:,:,:])
        NewMaskVolume.append(Maskvolume[8,:,:,:])
        NewMaskVolume.append(Maskvolume[1,:,:,:])

        Maskvolume = np.stack(NewMaskVolume, axis = 0)
        
        
        # ImageVolume = np.expand_dims(ImageVolume, axis=0)
        # print(Maskvolume.shape)
        Maskvolume = np.argmax(Maskvolume, axis=0)
        ImageVolume = np.transpose(ImageVolume, (1,2,0))
        Maskvolume = np.transpose(Maskvolume, (1,2,0))

        NewImageVolume = []
        NewMaskVolume = []

        for i in range(ImageVolume.shape[2]):
            Image2dSilce = self._preprocess_img(ImageVolume[:,:,i])
            Mask2dSlice = self._preprocess_mask(Maskvolume[:,:,i])
            # print(Image2dSilce.shape)
            NewImageVolume.append(Image2dSilce)
            NewMaskVolume.append(Mask2dSlice)

        ImageVolume = np.stack(NewImageVolume, axis=0)
        Maskvolume = np.stack(NewMaskVolume, axis=0)
        ImageVolume = np.squeeze(ImageVolume, axis=1)
        Maskvolume = np.squeeze(Maskvolume, axis=1)
        
        ImageVolume = torch.from_numpy(ImageVolume).float()
        Maskvolume = torch.from_numpy(Maskvolume).long()

        # ImageVolume = torch.tensor(ImageVolume, dtype=torch.float16)
        # Maskvolume = torch.tensor(Maskvolume, dtype=torch.float16)



        return ImageVolume, Maskvolume

In [103]:
data = CustomDataset(ImagesPath, MasksPath)

In [104]:
ImageVolume, Maskvolume = data[0]

2.25.285141789940989207961066447306753183932


In [105]:
ImageVolume.shape, Maskvolume.shape

(torch.Size([156, 512, 512]), torch.Size([156, 512, 512]))

In [106]:
np.unique(Maskvolume)

array([0, 2, 3])

In [107]:
net.eval()

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [108]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import torch
from torch.utils.data import DataLoader
from unet import UNet
from options.test_options import TestOptions
import torch
import torch.nn.functional as F

In [109]:
import torch.nn.functional as F

def dice_per_channel(output, target, epsilon=1e-6):
    dice_scores = []
    for c in range(output.shape[0]):
        pred_flat = output[c].reshape(-1)
        target_flat = target[c].reshape(-1)

        intersection = torch.sum(pred_flat * target_flat)
        union = torch.sum(pred_flat) + torch.sum(target_flat)

        dice = (2. * intersection + epsilon) / (union + epsilon)
        dice_scores.append(dice.item())
    
    return dice_scores

In [110]:
Total_dice_scores = [0,0,0,0,0,0]
for i in range(len(data)):
    ImageVolume, Maskvolume = data[i]
    for j in range(ImageVolume.shape[0]):
        image = ImageVolume[j, :,:]
        mask = Maskvolume[j,:,:]

        image = image.to(device=device, dtype=torch.float32)
        image = image.unsqueeze(0)
        image = image.unsqueeze(0)
    
        # mask_true = mask.to(device=device, dtype=torch.long)
        mask_true = mask.squeeze(0)
        mask_true = F.one_hot(mask_true, 6)
        mask_true = mask_true.permute(2, 0,1)
        # print(image.shape,'   ',mask_true.shape)
        with torch.no_grad():
            output = net(image).squeeze(0)
            output = (torch.sigmoid(output) > 0.5).float()
            output = output.to('cpu')
            
        
        # print(output.shape)
        # print(torch.unique(output))
        dice_scores = dice_per_channel(output, mask_true)
        # Total_dice_scores = Total_dice_scores + dice_scores
        Total_dice_scores = [a + b for a, b in zip(Total_dice_scores, dice_scores)]
        print(Total_dice_scores)

2.25.285141789940989207961066447306753183932
[0.9961110949516296, 1.0, 4.923683127877609e-10, 1.0, 1.0, 1.0]
[1.9961110949516296, 2.0, 1.0000000004923684, 2.0, 2.0, 2.0]
[2.9961110949516296, 3.0, 2.0000000004923684, 3.0, 3.0, 3.0]
[3.9961110949516296, 4.0, 3.0000000004923684, 4.0, 4.0, 4.0]
[4.99611109495163, 5.0, 4.000000000492369, 5.0, 5.0, 5.0]
[5.99611109495163, 6.0, 5.000000000492369, 6.0, 6.0, 6.0]
[6.991472005844116, 7.0, 5.0000000009054215, 7.0, 7.0, 7.0]
[7.991472005844116, 8.0, 6.0000000009054215, 8.0, 8.0, 8.0]
[8.991472005844116, 9.0, 7.0000000009054215, 9.0, 9.0, 9.0]
[9.991472005844116, 10.0, 8.000000000905422, 10.0, 10.0, 10.0]
[10.991472005844116, 11.0, 9.000000000905422, 11.0, 11.0, 11.0]
[11.987596571445465, 12.0, 9.000000001399492, 12.0, 12.0, 12.0]
[12.987596571445465, 13.0, 10.000000001399492, 13.0, 13.0, 13.0]
[13.987596571445465, 14.0, 11.000000001399492, 14.0, 14.0, 14.0]
[14.986395418643951, 15.0, 11.000000003780444, 14.000000004784688, 15.0, 15.0]
[15.98639541

In [111]:
numberofslice = 0
for i in range(len(data)):
    ImageVolume, Maskvolume = data[i]
    numberofslice = numberofslice + ImageVolume.shape[0]
    # print(ImageVolume.shape)

2.25.285141789940989207961066447306753183932
2.25.675643075030906466102858622863067480730
2.25.193204423187098024139005996142897510500
1.2.826.0.1.3680043.10.511.3.13027453861199312938709537287826696
2.25.958951608412750990375245965275148403553
2.25.969951689926195160380732867208688259675
2.25.160294964270049262155099322509120650489
2.25.963853606161210352739966258030989557592
2.25.150399916482666136350259459270412858780
2.25.221090026976406149882549881352828965267
2.25.550685872830491474237728954098007406082
2.25.575882319531989549479997009215637590207
2.25.632700076457296961281354238296421241038
2.25.928976977736828826873283737275918331792
2.25.519901969239213741570608404674777548364
1.2.826.0.1.3680043.10.511.3.11946135914663867924979387434002116
2.25.249857182000719120795252903989751385058
2.25.745322471434354940152329793278861630316
2.25.797456032886121937703228326787097519710
2.25.880761686993000194609688666381202837792
2.25.610396554128751564210682455152628389000
2.25.6131141454

In [112]:
numberofslice

26715

In [113]:
Total_dice_scores

[26693.90381771326,
 26274.00000159881,
 24409.000007124785,
 26151.000001177297,
 26392.000006198457,
 25841.000001048196]

In [114]:
Total_dice_scores[0]/numberofslice, Total_dice_scores[1]/numberofslice, Total_dice_scores[2]/numberofslice, Total_dice_scores[3]/numberofslice, Total_dice_scores[4]/numberofslice, Total_dice_scores[5]/numberofslice

(0.9992103244511795,
 0.9834924200486174,
 0.9136814526342798,
 0.9788882650637206,
 0.9879094144188081,
 0.9672842972505408)

In [116]:
classwise_dice = [Total_dice_scores[0]/numberofslice, Total_dice_scores[1]/numberofslice, Total_dice_scores[2]/numberofslice, Total_dice_scores[3]/numberofslice, Total_dice_scores[4]/numberofslice, Total_dice_scores[5]/numberofslice]

In [117]:
import numpy as np

def compute_confusion_matrix(y_true, y_pred):
    """
    y_true: ndarray of shape (C, D, H, W), binary (0 or 1)
    y_pred: ndarray of shape (C, D, H, W), binary (0 or 1)
    Returns:
        conf_matrix: (C, C) where entry (i, j) = true class i predicted as j
    """
    C = y_true.shape[0]
    conf_matrix = np.zeros((C, C), dtype=np.int64)

    for i in range(C):  # true class
        for j in range(C):  # predicted class
            # Count where class i is present in true and class j in pred
            intersection = (y_true[i] == 1) & (y_pred[j] == 1)
            conf_matrix[i, j] = np.sum(intersection)

    return conf_matrix

In [118]:
perclassDiceScore = {
    "Background":[],
    "Epidural Hemorrhage (EDH)": [],
    "Intraparenchymal Hemorrhage (IPH)": [],
    "Intraventricular Hemorrhage (IVH)": [],
    "Subarachnoid Hemorrhage (SAH)": [],
    "Subdural Hemorrhage (SDH)": [],
}
classesmaping = {
    0 : "Background",
    1 : "Epidural Hemorrhage (EDH)",
    2 : "Intraparenchymal Hemorrhage (IPH)",
    3 : "Intraventricular Hemorrhage (IVH)",
    4 : "Subarachnoid Hemorrhage (SAH)",
    5 : "Subdural Hemorrhage (SDH)",
}
conf_matrix = np.zeros((6, 6), dtype=np.int64)
for i in range(len(data)):
    img_data, msk_data = data[i]
    slicesCount = 32
    mul = 0
    while slicesCount*mul <= img_data.shape[2]:
        sub_img_data = img_data[:,:,slicesCount*mul:slicesCount*(mul+1)]
        sub_msk_data = msk_data[:,:,slicesCount*mul:slicesCount*(mul+1)]

        if sub_img_data.shape[2] >= 16:
            # transformed = train_transform(image=sub_img_data, mask=sub_msk_data)
            img_tensor = image
            orig_mask = mask_true
            #orig_mask = F.one_hot(orig_mask.long(), num_classes=6)
            #orig_mask = torch.permute(orig_mask, (3,2,0,1))
            print(orig_mask.shape, '  ', torch.unique(orig_mask))
            
            img_tensor = img_tensor.unsqueeze(0)
            img_tensor = img_tensor.unsqueeze(0)

            img_tensor = img_tensor.to(device)
            pred = net(img_tensor)
            pred = pred.squeeze(0)
            pred = torch.argmax(pred, axis=0)
            pred = F.one_hot(pred.long(), num_classes=6)
            pred = torch.permute(pred, (3, 0, 1, 2))
            pred = pred.cpu()

            print(pred.shape,'  ',torch.unique(pred))

            for i in range(6):
                DiceScore = classwise_dice[i]
                # DiceScore = dice_score_binary(pred[i,:,:,:], orig_mask[i,:,:,:])
                perclassDiceScore[classesmaping[i]].append(DiceScore)
                CF = compute_confusion_matrix(orig_mask.cpu().detach().numpy(), pred.cpu().detach().numpy())
                conf_matrix = conf_matrix + CF
                print(DiceScore)

            # DiceScore = dice_score_binary(pred, orig_mask)
            # print(DiceScore)
            # pred = pred.cpu().detach().numpy()
            
            # print(pred.shape,'  ',np.unique(pred))

            
        # print(img_data[:,:,slicesCount*mul:slicesCount*(mul+1)].shape)
        mul = mul + 1
    print(msk_data.shape)

2.25.285141789940989207961066447306753183932
torch.Size([6, 512, 512])    tensor([0, 1])


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 1, 1, 1, 512, 512]